In [3]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import math

In [20]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [21]:
link='https://str.optical.org/'

In [22]:
driver.get(link)

In [9]:
england_postcode_areas = [
'NE','YO','DL','CA','MI','S','B','L','E','N','TA','EX'
]

In [23]:
england_postcode_areas = [
'DL','CA','MI','S','B','L','E','N','TA','EX'
]

In [10]:
inputcodepostal=driver.find_element(By.ID,'Registrant-Geography-input')
clicksearch=driver.find_element(By.CSS_SELECTOR,'#searchform > div.card-footer > button.btn.btn-primary.float-right.mt-3')

In [14]:
slider = driver.find_element(By.ID, "BodyCorporate-Geography-range")

# Ejecuta JS para ponerlo en 100 y disparar el evento
driver.execute_script("""
    arguments[0].value = 100;
    arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
""", slider)



In [18]:
data1=[]
data2=[]
data3=[]
data4=[]

In [ ]:
rango = [5]

for area in england_postcode_areas:
    for district in rango:
        retries = 3
        while retries > 0:
            try:
           
                slider = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "Registrant-Geography-range"))
                )
                inputcodepostal = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'Registrant-Geography-input'))
                )
                clicksearch = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#searchform > div.card-footer > button.btn.btn-primary.float-right.mt-3'))
                )

                inputcodepostal.clear()
                inputcodepostal.send_keys(area + str(district))

  
                driver.execute_script("""
                    arguments[0].value = 100;
                    arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
                """, slider)

   
                clicksearch.click()
                time.sleep(2)

         
                soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                prepage = soup1.find('p', class_='d-block text-right mt-4')
                if not prepage:
                    raise ValueError("No se encontró el texto de número de páginas.")

                maxpage = int(prepage.text.strip().split()[-1])

                for numpage in range(1, maxpage + 1):
                    try:
     
                        inputpag = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.ID, 'goToPageInput'))
                        )
                        inputpag.clear()
                        inputpag.send_keys(numpage)
                        time.sleep(1)

                        searchbutton = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, '#search-results > div.card-body > div:nth-child(3) > div.col-md-4 > div > div > button'))
                        )
                        searchbutton.click()
                        time.sleep(1)

       
                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                        alltargets = soup2.find_all('li', class_='media mb-3 border-bottom pb-3')

                        for target in alltargets:
                            try:
                                predata1 = target.find('strong')
                                data1.append(predata1.text if predata1 else None)
                            except:
                                data1.append(None)

                            try:
                                predata2 = target.find('div', class_='media-body')
                                if predata2:
                                    lines = predata2.text.strip().split('\n')
                                    if lines:
                                        data2.append(lines[len(lines)-1].strip())
                                    else:
                                        data2.append(None)
                                else:
                                    data2.append(None)
                            except:
                                data2.append(None)
                            try:
                                predata3 = target.find('div', class_='media-body')
                                if predata3:
                                    lines = predata3.text.strip().split('\n')
                                    if lines:
                                        data3.append(lines[len(lines)-2].strip())
                                    else:
                                        data3.append(None)
                                else:
                                    data3.append(None)
                            except:
                                data3.append(None)
                            try:
                                predata4 = target.find('div', class_='media-body')
                                if predata4:
                                    lines = predata4.text.strip().split('\n')
                                    if lines:
                                        data4.append(lines[len(lines)-3].strip())
                                    else:
                                        data4.append(None)
                                else:
                                    data4.append(None)
                            except:
                                data4.append(None)
                    except Exception as e:
                        print(f"[!] Error en paginación página {numpage}: {e}")
                        continue
                break  
            except Exception as e:
                print(f"[!] Error con {area}{district}, reintentando... {3 - retries + 1}/3 - {e}")
                retries -= 1
                time.sleep(2)
        else:
            print(f"[x] Fallo definitivo en {area}{district}")

[!] Error con DL5, reintentando... 1/3 - No se encontró el texto de número de páginas.
[!] Error con DL5, reintentando... 2/3 - No se encontró el texto de número de páginas.
[!] Error con DL5, reintentando... 3/3 - No se encontró el texto de número de páginas.
[x] Fallo definitivo en DL5
[!] Error con CA5, reintentando... 1/3 - No se encontró el texto de número de páginas.
[!] Error con CA5, reintentando... 2/3 - No se encontró el texto de número de páginas.
[!] Error con CA5, reintentando... 3/3 - No se encontró el texto de número de páginas.
[x] Fallo definitivo en CA5
[!] Error con MI5, reintentando... 1/3 - No se encontró el texto de número de páginas.


KeyboardInterrupt: 

In [30]:
sample=({'Name':data1,
         'Adress':data2,
         'Speciality':data3,
         'Register_as':data4
         

})

In [31]:
sample=pd.DataFrame(sample)

In [33]:
sample.drop_duplicates(inplace=True)

In [36]:
sample['Speciality']=sample['Speciality'].apply(lambda x:x.replace('Specialities: ',''))

In [38]:
sample['Register_as']=sample['Register_as'].apply(lambda x:x.replace('Registered as: ',''))

In [40]:
sample.to_csv('stroptical.csv',index=False)
sample.to_excel('stroptical.xlsx',index=False)